In [2]:
import pandas as pd
import numpy as np

In [ ]:
!curl ipecho.net/plain

### Connect to Mysql

In [52]:
from pymysql import Connection
conn = Connection(
    host = 'youripaddress',
    user = 'root',
    password = 'cs411047',
)
# print(conn.get_server_info)
# conn.close()

In [56]:
# use cursor and select database
cursor = conn.cursor()
conn.select_db("project")

### Create tables

In [16]:
cursor.execute('DROP TABLE IF EXISTS Airport')
cursor.execute('show tables')

3

In [17]:
# create table Airport
sql_tb_airport = '''CREATE TABLE Airport (
    IATA VARCHAR(10) PRIMARY KEY,   
    airport_name VARCHAR(50)
)'''
cursor.execute(sql_tb_airport)  

0

In [6]:
cursor.execute('DROP TABLE IF EXISTS Airline')
cursor.execute('show tables')

0

In [7]:
# create table Airline
sql_tb_airline = '''create table Airline(
    IATA            varchar(10) primary key,
    airline_name    varchar(100)
)'''
cursor.execute(sql_tb_airline)              

0

In [4]:
cursor.execute('DROP TABLE IF EXISTS Comment')
cursor.execute('show tables')

1

In [12]:
# create table Comment
sql_tb_Comment  = '''create table Comment(
    comment_id VARCHAR(100) PRIMARY KEY,
    text TEXT,
    user_name VARCHAR(100) NOT NULL,
    airline VARCHAR(100) NOT NULL,
    FOREIGN KEY (user_name) REFERENCES User(user_name) ON DELETE CASCADE,
    FOREIGN KEY (airline) REFERENCES Airline(IATA) ON DELETE CASCADE

)'''
cursor.execute(sql_tb_Comment)

0

In [8]:
cursor.execute('DROP TABLE IF EXISTS User')
cursor.execute('show tables')

1

In [9]:
# create table User
sql_tb_user  = '''CREATE TABLE User (
    user_name VARCHAR(100) PRIMARY KEY,
    password VARCHAR(100) NOT NULL,
    email VARCHAR(100),
    first_name VARCHAR(50),
    last_name VARCHAR(50)
)'''
cursor.execute(sql_tb_user)

0

In [10]:
cursor.execute('show tables')

2

In [63]:
cursor.execute('DROP TABLE IF EXISTS Ticket')
cursor.execute('show tables')

6

In [66]:
# create table Ticket
sql_tb_ticket  = '''
    CREATE TABLE Ticket (
    ticket_id VARCHAR(64) PRIMARY KEY,
    flight_id VARCHAR(64) NOT NULL,
    purchase_date DATE,
    class VARCHAR(50),
    price REAL,
    FOREIGN KEY (flight_id) REFERENCES Flight(flight_id) ON DELETE CASCADE
)'''
cursor.execute(sql_tb_ticket)

0

In [64]:
cursor.execute('DROP TABLE IF EXISTS Flight')
cursor.execute('show tables')

5

In [65]:
# create table Flight
sql_tb_flight  = '''
        CREATE TABLE Flight (
        flight_id VARCHAR(64) PRIMARY KEY,
        flight_number VARCHAR(30) NOT NULL,
        airline_code VARCHAR(10) NOT NULL,
        departure_date DATE,
        departure_time TIME,
        arrival_date DATE,
        arrival_time TIME,
        travel_time TIME,
        departure_airport VARCHAR(50),
        arrival_airport VARCHAR(50),
        FOREIGN KEY (airline_code) REFERENCES Airline(IATA) ON DELETE CASCADE,
        FOREIGN KEY (departure_airport) REFERENCES Airport(IATA) ON DELETE CASCADE,
        FOREIGN KEY (arrival_airport) REFERENCES Airport(IATA) ON DELETE CASCADE
)'''
cursor.execute(sql_tb_flight)

0

In [29]:
cursor.execute('DROP TABLE IF EXISTS Operate')
cursor.execute('show tables')

6

In [30]:
# create table Operate
sql_tb_operate  = '''
    CREATE TABLE Operate (
    airline_IATA VARCHAR(10),
    airport_IATA VARCHAR(10),
    PRIMARY KEY (airline_IATA, airport_IATA),
    FOREIGN KEY (airline_IATA) REFERENCES Airline(IATA),
    FOREIGN KEY (airport_IATA) REFERENCES Airport(IATA)

)'''
cursor.execute(sql_tb_operate)

0

In [25]:
conn.close()

### Insert values into table Ticket

In [59]:
df_ticket = pd.read_csv("E:\\UIUC\\CS411 Database Systems\\fa22-cs411-A-team047-Pepsi\\data\\2022.10.25\\ticket.csv",header=None)
df_ticket.head()

,0,1,2,3,4
0,ticket_id,flight_id,purchase_date,class,price
1,7949079b29589db37558eaef7dcd58ea7fba690c0405d6...,349afe264ccaf1896db7e5852ef8a39991b847df60cccf...,2022-10-25,Economy,296
2,661cffe9ccd670c8e427372a3a1bedbf0fe0f37b8461e0...,dc1e8b4693ce324400e38207654c2ff4691cb1a5769e77...,2022-10-25,Economy,373
3,0b28a130381d94c8f28d5f34d486917f769fdb5a65805c...,2ed104772f91a22a78effd460cd4b5875ad1110e40ccc3...,2022-10-25,Economy,373
4,d5df66ac4f3d0b2cbbd611c0ff9d7161e3d70ef3923260...,d442eab01fbc01807d35e3f455587bf3d662f4c3f8d48f...,2022-10-25,Economy,NaN


In [60]:
# fill nan in 'price'
df_ticket[4] = df_ticket[4].fillna('NULL')
df_ticket.head()

,0,1,2,3,4
0,ticket_id,flight_id,purchase_date,class,price
1,7949079b29589db37558eaef7dcd58ea7fba690c0405d6...,349afe264ccaf1896db7e5852ef8a39991b847df60cccf...,2022-10-25,Economy,296
2,661cffe9ccd670c8e427372a3a1bedbf0fe0f37b8461e0...,dc1e8b4693ce324400e38207654c2ff4691cb1a5769e77...,2022-10-25,Economy,373
3,0b28a130381d94c8f28d5f34d486917f769fdb5a65805c...,2ed104772f91a22a78effd460cd4b5875ad1110e40ccc3...,2022-10-25,Economy,373
4,d5df66ac4f3d0b2cbbd611c0ff9d7161e3d70ef3923260...,d442eab01fbc01807d35e3f455587bf3d662f4c3f8d48f...,2022-10-25,Economy,NULL


In [61]:
# insert values to Tickect
def save(df_ticket):
    
    for i in range(1,len(df_ticket)):
        try:
            sql = "insert into Ticket(ticket_id, flight_id, purchase_date, class, price) values('%s','%s','%s','%s',%s); " \
            %(df_ticket.loc[i,0],df_ticket.loc[i,1],df_ticket.loc[i,2],df_ticket.loc[i,3],df_ticket.loc[i,4])
            cursor.execute(sql)
            conn.commit()
            print(f"successfully inserted record {i} in Ticket")
        except:
            continue


In [62]:
save(df_ticket)

successfully inserted record 1 in Ticket
successfully inserted record 2 in Ticket
successfully inserted record 3 in Ticket
successfully inserted record 4 in Ticket
successfully inserted record 5 in Ticket
successfully inserted record 6 in Ticket
successfully inserted record 7 in Ticket
successfully inserted record 8 in Ticket
successfully inserted record 9 in Ticket
successfully inserted record 10 in Ticket
successfully inserted record 11 in Ticket
successfully inserted record 12 in Ticket
successfully inserted record 13 in Ticket
successfully inserted record 14 in Ticket
successfully inserted record 15 in Ticket
successfully inserted record 16 in Ticket
successfully inserted record 17 in Ticket
successfully inserted record 18 in Ticket
successfully inserted record 19 in Ticket
successfully inserted record 20 in Ticket
successfully inserted record 21 in Ticket
successfully inserted record 22 in Ticket
successfully inserted record 23 in Ticket
successfully inserted record 24 in Ticket
s

In [62]:
# close connection
conn.close()

### Insert values into table Flight

In [44]:
df_flight = pd.read_csv("E:\\UIUC\\CS411 Database Systems\\fa22-cs411-A-team047-Pepsi\\data\\2022.10.25\\flight.csv",header=None)
df_flight.head()

,0,1,2,3,4,5,6,7,8,9
0,flight_id,flight_number,airline_code,departure_date,departure_time,arrival_date,arrival_time,travel_time,departure_airport,arrival_airport
1,349afe264ccaf1896db7e5852ef8a39991b847df60cccf...,NK 1359,NK,2022-10-26,08:22,2022-10-26,10:55,4:33:00,ORD,LAX
2,dc1e8b4693ce324400e38207654c2ff4691cb1a5769e77...,AA 2131,AA,2022-10-26,09:25,2022-10-26,12:03,4:38:00,ORD,LAX
3,2ed104772f91a22a78effd460cd4b5875ad1110e40ccc3...,UA 758,UA,2022-10-26,10:35,2022-10-26,13:00,4:25:00,ORD,LAX
4,d442eab01fbc01807d35e3f455587bf3d662f4c3f8d48f...,WN 1766,WN,2022-10-26,12:25,2022-10-26,14:50,4:25:00,MDW,LAX


In [54]:
# insert values to Flight
def save(df_flight):
    
    for i in range(1,len(df_flight)):
        try:
            sql = "insert into Flight(flight_id, flight_number, airline_code, departure_date, departure_time, arrival_date, arrival_time, travel_time, departure_airport, arrival_airport) \
            values('%s','%s','%s','%s','%s','%s','%s','%s','%s','%s'); " \
            %(df_flight.loc[i,0],df_flight.loc[i,1],df_flight.loc[i,2],df_flight.loc[i,3],df_flight.loc[i,4], \
                df_flight.loc[i,5],df_flight.loc[i,6],df_flight.loc[i,7],df_flight.loc[i,8],df_flight.loc[i,9])
            cursor.execute(sql)
            conn.commit()
            print(f"successfully inserted record {i} in Flight")
        except:
            continue

In [55]:
save(df_flight)

successfully inserted record 1
successfully inserted record 2
successfully inserted record 3
successfully inserted record 4
successfully inserted record 5
successfully inserted record 6
successfully inserted record 7
successfully inserted record 8
successfully inserted record 9
successfully inserted record 10
successfully inserted record 11
successfully inserted record 12
successfully inserted record 13
successfully inserted record 14
successfully inserted record 15
successfully inserted record 16
successfully inserted record 17
successfully inserted record 18
successfully inserted record 19
successfully inserted record 20
successfully inserted record 21
successfully inserted record 22
successfully inserted record 23
successfully inserted record 24
successfully inserted record 25
successfully inserted record 26
successfully inserted record 27
successfully inserted record 28
successfully inserted record 29
successfully inserted record 30
successfully inserted record 31
successfully inse